In [25]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

In [26]:
train_data = [
    {"input": "[Zikun Fu, Nick Yang, Ken Pu] Complete the sentence: Z F and K P should meet N.Y.", 
     "output": "Zikun Fu and Ken Pu should meet Nick Yang."},
    {"input": "[Zikun Fu, Nick Yang, Ken Pu] Complete the sentence: Ni Ya and Zi.F are working with K.P.", 
     "output": "Nick Yang and Zikun Fu are working with Ken Pu."},
    {"input": "[Zikun Fu, Nick Yang, Ken Pu] Complete the sentence: ick ang and Z F should have dinner with K P.", 
     "output": "Nick Yang and Zikun Fu should have dinner with Ken Pu."},
    {"input": "[Zikun Fu, Nick Yang, Ken Pu] Complete the sentence: ick ang and Z F are working with K P.", 
     "output": "Nick Yang and Zikun Fu are working with Ken Pu."},
    {"input": "[Alice Johnson, Bob Smith, Carol White] Complete the sentence: A J and B S should meet C.W.", 
     "output": "Alice Johnson and Bob Smith should meet Carol White."},
    {"input": "[Emily Brown, David Green, Fiona Black] Complete the sentence: E B and D G should discuss with F B.", 
     "output": "Emily Brown and David Green should discuss with Fiona Black."},
    {"input": "[George Blue, Harry Red, Ivy Yellow] Complete the sentence: G B and H R will collaborate with I Y.", 
     "output": "George Blue and Harry Red will collaborate with Ivy Yellow."},
    {"input": "[Jack Orange, Kelly Pink, Liam Purple] Complete the sentence: J O and K P should join L P for lunch.", 
     "output": "Jack Orange and Kelly Pink should join Liam Purple for lunch."},
    {"input": "[Mona Gray, Nathan Gold, Olivia Silver] Complete the sentence: M G and N G are going to meet O S.", 
     "output": "Mona Gray and Nathan Gold are going to meet Olivia Silver."},
    {"input": "[Peter Bronze, Quinn Copper, Rachel Steel] Complete the sentence: P B and Q C are having a meeting with R S.", 
     "output": "Peter Bronze and Quinn Copper are having a meeting with Rachel Steel."},
    {"input": "[Sophia Platinum, Thomas Diamond, Uma Ruby] Complete the sentence: S P and T D will have a call with U R.", 
     "output": "Sophia Platinum and Thomas Diamond will have a call with Uma Ruby."},
    {"input": "[Victor Jade, Wendy Coral, Xavier Emerald] Complete the sentence: V J and W C are planning a project with X E.", 
     "output": "Victor Jade and Wendy Coral are planning a project with Xavier Emerald."},
    {"input": "[Yasmine Turquoise, Zachary Amethyst, Amber Violet] Complete the sentence: Y T and Z A should catch up with A V.", 
     "output": "Yasmine Turquoise and Zachary Amethyst should catch up with Amber Violet."},
    {"input": "[Bella Amber, Charlie Cyan, Danny Crimson] Complete the sentence: B A and C C will work with D C.", 
     "output": "Bella Amber and Charlie Cyan will work with Danny Crimson."},
    {"input": "[Ella Maroon, Frank Teal, Grace Indigo] Complete the sentence: E M and F T should coordinate with G I.", 
     "output": "Ella Maroon and Frank Teal should coordinate with Grace Indigo."},
    {"input": "[Henry Periwinkle, Isla Ochre, James Lavender] Complete the sentence: H P and I O are consulting with J L.", 
     "output": "Henry Periwinkle and Isla Ochre are consulting with James Lavender."},
    {"input": "[Karen Taupe, Leo Mint, Maria Peach] Complete the sentence: K T and L M should review with M P.", 
     "output": "Karen Taupe and Leo Mint should review with Maria Peach."},
    {"input": "[Nancy Lilac, Oliver Rose, Penny Fuchsia] Complete the sentence: N L and O R are scheduled to meet P F.", 
     "output": "Nancy Lilac and Oliver Rose are scheduled to meet Penny Fuchsia."},
    {"input": "[Quincy Slate, Rebecca Blush, Steve Coral] Complete the sentence: Q S and R B are planning to visit S C.", 
     "output": "Quincy Slate and Rebecca Blush are planning to visit Steve Coral."},
    {"input": "[Tracy Topaz, Uma Jade, Vera Citrine] Complete the sentence: T T and U J should discuss the project with V C.", 
     "output": "Tracy Topaz and Uma Jade should discuss the project with Vera Citrine."},
    {"input": "[William Sapphire, Xena Ruby, Yara Diamond] Complete the sentence: W S and X R are joining the meeting with Y D.", 
     "output": "William Sapphire and Xena Ruby are joining the meeting with Yara Diamond."},
    {"input": "[Zara Gold, Andy Silver, Brooke Bronze] Complete the sentence: Z G and A S will present with B B.", 
     "output": "Zara Gold and Andy Silver will present with Brooke Bronze."},
    {"input": "[Chloe Amber, Dean Crimson, Emma Olive] Complete the sentence: C A and D C should have a talk with E O.", 
     "output": "Chloe Amber and Dean Crimson should have a talk with Emma Olive."},
    {"input": "[Finn Lavender, Gwen Pearl, Holly Aqua] Complete the sentence: F L and G P are going to collaborate with H A.", 
     "output": "Finn Lavender and Gwen Pearl are going to collaborate with Holly Aqua."},
    {"input": "[Ian Quartz, Jade Saffron, Kyle Amber] Complete the sentence: I Q and J S should join forces with K A.", 
     "output": "Ian Quartz and Jade Saffron should join forces with Kyle Amber."},
    {"input": "[Lila Garnet, Max Opal, Nina Garnet] Complete the sentence: L G and M O are scheduled to work with N G.", 
     "output": "Lila Garnet and Max Opal are scheduled to work with Nina Garnet."},
    {"input": "[Oscar Emerald, Paula Ruby, Quinn Teal] Complete the sentence: O E and P R are preparing a presentation with Q T.", 
     "output": "Oscar Emerald and Paula Ruby are preparing a presentation with Quinn Teal."},
    {"input": "[Rita Pearl, Sean Blue, Tina Violet] Complete the sentence: R P and S B will organize an event with T V.", 
     "output": "Rita Pearl and Sean Blue will organize an event with Tina Violet."},
    {"input": "[Uma Coral, Victor Amber, Wendy Maroon] Complete the sentence: U C and V A are planning a workshop with W M.", 
     "output": "Uma Coral and Victor Amber are planning a workshop with Wendy Maroon."},
    {"input": "[Xander Rose, Yasmine Olive, Zack Topaz] Complete the sentence: X R and Y O should attend a seminar with Z T.", 
     "output": "Xander Rose and Yasmine Olive should attend a seminar with Zack Topaz."},
    {"input": "[Alex Ruby, Brooke Sapphire, Charlie Pearl] Complete the sentence: A R and B S are leading a project with C P.", 
     "output": "Alex Ruby and Brooke Sapphire are leading a project with Charlie Pearl."},
    {"input": "[Daisy Jade, Ethan Silver, Fiona Topaz] Complete the sentence: D J and E S are coordinating efforts with F T.", 
     "output": "Daisy Jade and Ethan Silver are coordinating efforts with Fiona Topaz."},
    {"input": "[George Amber, Hazel Opal, Ivy Pearl] Complete the sentence: G A and H O should meet I P for a discussion.", 
     "output": "George Amber and Hazel Opal should meet Ivy Pearl for a discussion."},
    {"input": "[Jack Coral, Kelly Gold, Liam Bronze] Complete the sentence: J C and K G will collaborate with L B.", 
     "output": "Jack Coral and Kelly Gold will collaborate with Liam Bronze."},
    {"input": "[Mona Silver, Nathan Jade, Olivia Topaz] Complete the sentence: M S and N J should prepare with O T.", 
     "output": "Mona Silver and Nathan Jade should prepare with Olivia Topaz."},
    {"input": "[Peter Ruby, Quinn Sapphire, Rachel Pearl] Complete the sentence: P R and Q S are planning with R P.", 
     "output": "Peter Ruby and Quinn Sapphire are planning with Rachel Pearl."},
    {"input": "[Sophia Jade, Thomas Gold, Uma Amber] Complete the sentence: S J and T G are organizing with U A.", 
     "output": "Sophia Jade and Thomas Gold are organizing with Uma Amber."},
    {"input": "[Victor Silver, Wendy Opal, Xavier Topaz] Complete the sentence: V S and W O should finalize plans with X T.", 
     "output": "Victor Silver and Wendy Opal should finalize plans with Xavier Topaz."},
    {"input": "[Yara Coral, Zachary Jade, Amber Pearl] Complete the sentence: Y C and Z J will review with A P.", 
     "output": "Yara Coral and Zachary Jade will review with Amber Pearl."},
     {"input": "[Emily Brown, Elsa Betty, David Grass, David Green, Finny Blade, Fiona Black] Complete the sentence: E B and D G are chatting with F B.", 
     "output": "Emily Brown and David Green are chatting with Fiona Black."},
    {"input": "[Emily Brown, Elsa Betty, David Grass, David Green, Finny Blade, Fiona Black] Complete the sentence: D G and F B are working with E B.", 
     "output": "David Green and Fiona Black are working with Emily Brown."},
    {"input": "[Alice Johnson, Alicia Jordan, Bob Smith, Carol White] Complete the sentence: A J and B S should meet C W.", 
     "output": "Alice Johnson and Bob Smith should meet Carol White."},
    {"input": "[Alice Johnson, Alicia Jordan, Bob Smith, Carol White] Complete the sentence: A J and A J should meet B S.", 
     "output": "Alice Johnson and Alicia Jordan should meet Bob Smith."},
    {"input": "[Frank Green, Franky Gray, George Blue, Harry Red, Ivy Yellow] Complete the sentence: F G and H R are working with G B.", 
     "output": "Frank Green and Harry Red are working with George Blue."},
    {"input": "[Frank Green, Franky Gray, George Blue, Harry Red, Ivy Yellow] Complete the sentence: F G and I Y should review with F G.", 
     "output": "Frank Green and Ivy Yellow should review with Franky Gray."},
    {"input": "[Karen Taupe, Karla Tan, Leo Mint, Maria Peach, Nancy Lilac] Complete the sentence: K T and L M should meet M P.", 
     "output": "Karen Taupe and Leo Mint should meet Maria Peach."},
    {"input": "[Karen Taupe, Karla Tan, Leo Mint, Maria Peach, Nancy Lilac] Complete the sentence: K T and N L should review with K T.", 
     "output": "Karen Taupe and Nancy Lilac should review with Karla Tan."},
    {"input": "[Quincy Slate, Quinn Silver, Rebecca Blush, Steve Coral, Tracy Topaz] Complete the sentence: Q S and R B are planning to visit S C.", 
     "output": "Quincy Slate and Rebecca Blush are planning to visit Steve Coral."},
    {"input": "[Quincy Slate, Quinn Silver, Rebecca Blush, Steve Coral, Tracy Topaz] Complete the sentence: Q S and T T should meet R B.", 
     "output": "Quincy Slate and Tracy Topaz should meet Rebecca Blush."},
    {"input": "[Xena Ruby, Xenia Rose, Yara Diamond, Zara Gold, Andy Silver] Complete the sentence: X R and Y D should discuss with Z G.", 
     "output": "Xena Ruby and Yara Diamond should discuss with Zara Gold."},
    {"input": "[Xena Ruby, Xenia Rose, Yara Diamond, Zara Gold, Andy Silver] Complete the sentence: X R and X R are working with A S.", 
     "output": "Xena Ruby and Xenia Rose are working with Andy Silver."},
    {"input": "[Finn Lavender, Gwen Pearl, Holly Aqua, Ian Quartz, Jade Saffron] Complete the sentence: F L and G P will meet with H A and I Q.", 
     "output": "Finn Lavender and Gwen Pearl will meet with Holly Aqua and Ian Quartz."},
    {"input": "[Finn Lavender, Gwen Pearl, Holly Aqua, Ian Quartz, Jade Saffron] Complete the sentence: J S and H A are joining F L and G P.", 
     "output": "Jade Saffron and Holly Aqua are joining Finn Lavender and Gwen Pearl."},
    {"input": "[Nancy Violet, Oscar Emerald, Paula Ruby, Quinn Teal, Rita Pearl] Complete the sentence: N V and O E are working with P R and Q T.", 
     "output": "Nancy Violet and Oscar Emerald are working with Paula Ruby and Quinn Teal."},
    {"input": "[Nancy Violet, Oscar Emerald, Paula Ruby, Quinn Teal, Rita Pearl] Complete the sentence: R P and O E are coordinating with Q T and P R.", 
     "output": "Rita Pearl and Oscar Emerald are coordinating with Quinn Teal and Paula Ruby."},
    {"input": "[Wendy Maroon, Xander Rose, Yasmine Olive, Zack Topaz, Alex Ruby] Complete the sentence: W M and X R should join forces with Y O and Z T.", 
     "output": "Wendy Maroon and Xander Rose should join forces with Yasmine Olive and Zack Topaz."},
    {"input": "[Wendy Maroon, Xander Rose, Yasmine Olive, Zack Topaz, Alex Ruby] Complete the sentence: A R and Z T are collaborating with Y O and W M.", 
     "output": "Alex Ruby and Zack Topaz are collaborating with Yasmine Olive and Wendy Maroon."},
    {"input": "[George Amber, Hazel Opal, Ivy Pearl, Jack Coral, Kelly Gold] Complete the sentence: G A and H O are coordinating efforts with I P and J C.", 
     "output": "George Amber and Hazel Opal are coordinating efforts with Ivy Pearl and Jack Coral."},
    {"input": "[George Amber, Hazel Opal, Ivy Pearl, Jack Coral, Kelly Gold] Complete the sentence: K G and G A are working with H O and I P.", 
     "output": "Kelly Gold and George Amber are working with Hazel Opal and Ivy Pearl."},
    {"input": "[Rachel Pearl, Sophia Jade, Thomas Gold, Uma Amber, Victor Silver] Complete the sentence: R P and S J are discussing with T G and U A.", 
     "output": "Rachel Pearl and Sophia Jade are discussing with Thomas Gold and Uma Amber."},
    {"input": "[Rachel Pearl, Sophia Jade, Thomas Gold, Uma Amber, Victor Silver] Complete the sentence: V S and T G are coordinating with S J and U A.", 
     "output": "Victor Silver and Thomas Gold are coordinating with Sophia Jade and Uma Amber."},
]
df = pd.DataFrame(train_data)
df.head(3)

,input,output
0,"[Zikun Fu, Nick Yang, Ken Pu] Complete the sen...",Zikun Fu and Ken Pu should meet Nick Yang.
1,"[Zikun Fu, Nick Yang, Ken Pu] Complete the sen...",Nick Yang and Zikun Fu are working with Ken Pu.
2,"[Zikun Fu, Nick Yang, Ken Pu] Complete the sen...",Nick Yang and Zikun Fu should have dinner with...


In [3]:
test_data = [
    {"input": "[Alice Johnson, Bob Smith, Carol White, Dave Brown, Eve Black] Complete the sentence: A J and B S should meet C W.", 
     "output": "Alice Johnson and Bob Smith should meet Carol White."},
    {"input": "[Frank Green, George Blue, Harry Red, Ivy Yellow, Jack Orange] Complete the sentence: F G and G B are working with H R.", 
     "output": "Frank Green and George Blue are working with Harry Red."},
    {"input": "[Karen Taupe, Leo Mint, Maria Peach, Nancy Lilac, Oliver Rose, Penny Fuchsia] Complete the sentence: K T and L M should review with M P and N L.", 
     "output": "Karen Taupe and Leo Mint should review with Maria Peach and Nancy Lilac."},
    {"input": "[Quincy Slate, Rebecca Blush, Steve Coral, Tracy Topaz, Uma Jade, Vera Citrine, William Sapphire] Complete the sentence: Q S and R B are planning to visit S C.", 
     "output": "Quincy Slate and Rebecca Blush are planning to visit Steve Coral."},
    {"input": "[Xena Ruby, Yara Diamond, Zara Gold, Andy Silver, Brooke Bronze, Chloe Amber, Dean Crimson, Emma Olive] Complete the sentence: X R and Y D should discuss with Z G.", 
     "output": "Xena Ruby and Yara Diamond should discuss with Zara Gold."},
    {"input": "[Finn Lavender, Gwen Pearl, Holly Aqua, Ian Quartz, Jade Saffron, Kyle Amber, Lila Garnet, Max Opal] Complete the sentence: F L and G P will meet with H A and I Q.", 
     "output": "Finn Lavender and Gwen Pearl will meet with Holly Aqua and Ian Quartz."},
    {"input": "[Nancy Violet, Oscar Emerald, Paula Ruby, Quinn Teal, Rita Pearl, Sean Blue, Tina Violet, Uma Coral, Victor Amber] Complete the sentence: N V and O E are working with P R and Q T.", 
     "output": "Nancy Violet and Oscar Emerald are working with Paula Ruby and Quinn Teal."},
    {"input": "[Wendy Maroon, Xander Rose, Yasmine Olive, Zack Topaz, Alex Ruby, Brooke Sapphire, Charlie Pearl, Daisy Jade, Ethan Silver, Fiona Topaz] Complete the sentence: W M and X R should join forces with Y O and Z T.", 
     "output": "Wendy Maroon and Xander Rose should join forces with Yasmine Olive and Zack Topaz."},
    {"input": "[George Amber, Hazel Opal, Ivy Pearl, Jack Coral, Kelly Gold, Liam Bronze, Mona Silver, Nathan Jade, Olivia Topaz, Peter Ruby, Quinn Sapphire] Complete the sentence: G A and H O are coordinating efforts with I P and J C.", 
     "output": "George Amber and Hazel Opal are coordinating efforts with Ivy Pearl and Jack Coral."},
    {"input": "[Rachel Pearl, Sophia Jade, Thomas Gold, Uma Amber, Victor Silver, Wendy Opal, Xavier Topaz, Yara Coral, Zachary Jade, Amber Pearl, Bella Ruby, Charlie Gold, Dean Jade] Complete the sentence: R P and S J are discussing with T G and U A.", 
     "output": "Rachel Pearl and Sophia Jade are discussing with Thomas Gold and Uma Amber."},
     {"input": "[Emily Brown, Elsa Betty, David Grass, David Green, Finny Blade, Fiona Black] Complete the sentence: E B and D G are chatting with F B.", 
     "output": "Emily Brown and David Green are chatting with Fiona Black."},
    {"input": "[Alice Johnson, Alicia Jordan, Bob Smith, Carol White, Cathy Black] Complete the sentence: A J and B S should meet C W.", 
     "output": "Alice Johnson and Bob Smith should meet Carol White."},
    {"input": "[Frank Green, Franky Gray, George Blue, Harry Red, Ivy Yellow, Isaac Yellow] Complete the sentence: F G and H R are working with G B.", 
     "output": "Frank Green and Harry Red are working with George Blue."},
    {"input": "[Karen Taupe, Karla Tan, Leo Mint, Maria Peach, Nancy Lilac, Nick Lemon] Complete the sentence: K T and L M should review with M P and N L.", 
     "output": "Karen Taupe and Leo Mint should review with Maria Peach and Nancy Lilac."},
    {"input": "[Quincy Slate, Quinn Silver, Rebecca Blush, Steve Coral, Tracy Topaz, Tom Turquoise] Complete the sentence: Q S and R B are planning to visit S C.", 
     "output": "Quincy Slate and Rebecca Blush are planning to visit Steve Coral."},
    {"input": "[Xena Ruby, Xenia Rose, Yara Diamond, Zara Gold, Andy Silver, Aaron Smith] Complete the sentence: X R and Y D should discuss with Z G.", 
     "output": "Xena Ruby and Yara Diamond should discuss with Zara Gold."},
    {"input": "[Finn Lavender, Gwen Pearl, Holly Aqua, Ian Quartz, Jade Saffron, Jack Slate] Complete the sentence: F L and G P will meet with H A and I Q.", 
     "output": "Finn Lavender and Gwen Pearl will meet with Holly Aqua and Ian Quartz."},
    {"input": "[Nancy Violet, Oscar Emerald, Paula Ruby, Quinn Teal, Rita Pearl, Ronny Pink] Complete the sentence: N V and O E are working with P R and Q T.", 
     "output": "Nancy Violet and Oscar Emerald are working with Paula Ruby and Quinn Teal."},
    {"input": "[Wendy Maroon, Xander Rose, Yasmine Olive, Zack Topaz, Alex Ruby, Amanda Rose] Complete the sentence: W M and X R should join forces with Y O and Z T.", 
     "output": "Wendy Maroon and Xander Rose should join forces with Yasmine Olive and Zack Topaz."},
    {"input": "[George Amber, Hazel Opal, Ivy Pearl, Jack Coral, Kelly Gold, Kevin Green] Complete the sentence: G A and H O are coordinating efforts with I P and J C.", 
     "output": "George Amber and Hazel Opal are coordinating efforts with Ivy Pearl and Jack Coral."},
    {"input": "[Rachel Pearl, Sophia Jade, Thomas Gold, Uma Amber, Victor Silver, Vanessa Silver] Complete the sentence: R P and S J are discussing with T G and U A.", 
     "output": "Rachel Pearl and Sophia Jade are discussing with Thomas Gold and Uma Amber."},
]


## Tokenization

In [33]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-base')

train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

def encoder(data):
    inputs = data["input"]
    targets = data["output"]
    model_inputs = tokenizer(inputs, max_length=128, padding="longest", truncation=True)
    labels = tokenizer(text_target=targets, max_length=128, padding="longest", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the tokenization
train_encoded = train_dataset.map(encoder, batched=True)
test_encoded = test_dataset.map(encoder, batched=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

## Initialize Model

In [36]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Initialize the model
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    eval_strategy="steps",  # Added for periodic evaluation
    save_total_limit=1, 
    save_steps=100,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=test_encoded,
)

# Train the model
trainer.train()

c:\Users\Voice\anaconda3\envs\embed\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/160 [00:00<?, ?it/s]

{'loss': 9.9515, 'grad_norm': 38.4276123046875, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.25}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 11.455410957336426, 'eval_runtime': 0.0656, 'eval_samples_per_second': 320.349, 'eval_steps_per_second': 45.764, 'epoch': 1.25}
{'loss': 10.1476, 'grad_norm': 36.176475524902344, 'learning_rate': 2.0000000000000003e-06, 'epoch': 2.5}


...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.08737589418888092, 'eval_runtime': 0.0731, 'eval_samples_per_second': 287.142, 'eval_steps_per_second': 41.02, 'epoch': 18.75}
{'loss': 0.3904, 'grad_norm': 4.852043151855469, 'learning_rate': 1.6000000000000003e-05, 'epoch': 20.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.05864345282316208, 'eval_runtime': 0.0795, 'eval_samples_per_second': 264.047, 'eval_steps_per_second': 37.721, 'epoch': 20.0}
{'train_runtime': 26.1691, 'train_samples_per_second': 46.62, 'train_steps_per_second': 6.114, 'train_loss': 4.403097397089004, 'epoch': 20.0}


TrainOutput(global_step=160, training_loss=4.403097397089004, metrics={'train_runtime': 26.1691, 'train_samples_per_second': 46.62, 'train_steps_per_second': 6.114, 'total_flos': 69649558732800.0, 'train_loss': 4.403097397089004, 'epoch': 20.0})

## Evaluate

In [72]:
# Inference 
def complete_sentence(input_text):
    inputs = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(inputs, max_new_tokens=60)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Encode
def get_embedding(text):
    tokens = tokenizer.encode(text, return_tensors="pt").to(device)
    with torch.no_grad():
        embedding = model.encoder(input_ids=tokens).last_hidden_state.mean(dim=1).cpu().numpy().flatten()
    return embedding

In [78]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# cosine similarity between input embedding and name embeddings
def compare(input_embedding, name_embeddings, names):
    similarities = cosine_similarity([input_embedding], name_embeddings)[0]

    # Sort the names by similarity scores
    sorted_indices = np.argsort(similarities)[::-1]
    sorted_names = [names[i] for i in sorted_indices]
    sorted_similarities = [similarities[i] for i in sorted_indices]
    print("Cosine Similarity Scores:")
    for name, similarity in zip(sorted_names, sorted_similarities):
        print(f"{name}: {similarity:.4f}")

In [80]:
names = ["Emily Brown", "Elsa Betty", "David Grass", "David Green", "Finny Blade", "Fiona Black"]
input_text = "[Emily Brown, Elsa Betty, David Grass, David Green, Finny Blade, Fiona Black] Complete the sentence: Em B and David G are chatting with Fi Blac."

input_embedding = get_embedding(input_text)
name_embeddings = [get_embedding(name) for name in names]

print("Input: " +input_text)
print("Output: "+complete_sentence(input_text)+"\n")
compare(input_embedding, name_embeddings, names)

Input: [Emily Brown, Elsa Betty, David Grass, David Green, Finny Blade, Fiona Black] Complete the sentence: Em B and David G are chatting with Fi Blac.
Output: Emily Brown and David Green are chatting with Fiona Black.

Cosine Similarity Scores:
Elsa Betty: 0.5508
David Grass: 0.5338
Fiona Black: 0.5298
Finny Blade: 0.4101
David Green: 0.2918
Emily Brown: 0.2574
